In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
data = pd.read_csv('./data/diabetes_012_health_indicators_BRFSS2015.csv')

1. Flujo de Entrenamiento:
    * Cross Validation
    * Boostrapping
    * Whole Training Dataset

2. Test - Predict
3. Model EValuation
    * Graphs
    * Kruskal Wallis

## Esquema de entrenamiento

Para el entrenamiento de los diferentes modelos de clasificación a emplear, se seguirá el siguiente flujo de entrenamiento: 

1. **Se separan los datos en dos grupos**: Un bloque de datos será destinado para el entrenamiento de los modelos y otro para su evaluación, en una distribución de 75/25. Donde los datos que sean empleados para entrenar el modelo no serán los mismos que los empleados para poder evaluarlo, evitando de esta forma brindar conclusiones únicamente de datos ya previamente conocidos por los diferentes clasificadores. Por ende, ninguno de los datos presentes en el bloque de entrenamiento será empleado en el set de datos empleados para la evaluación de los modelos.
2. **Stratified K-Fold Cross Validation**: Se dividirán los datos de entrenamiento en diferentes particiones para poder entrenar el modelo, evitar una sobredependencia en los datos ya vistos y evitar de esta forma que los modelos se sobreajusten (*over-fitting*) a la información y tengan un muy buen rendimiento aparente en datos conocidos pero al momento de agregar o probar con nueva información este sea incapaz de procesarla o lo haga con resultados pobres.

    Se empleará la variente de Particiones Estratificadas o Stratified K-Fold, debido a que se esta trabajando con un conjunto de datos desbalanceado, donde además de ser múlticlase, las clases se encuentran distribuidas de forma desigual. En este caso, como se mencionó anteriormente, existe una sobre-representación por parte de los pacientes saludables en comparación con los diabéticos y los pre-diabéticos, y si no se toman medidas para poder ajustar los modelos a estas variaciones en las representaciones de los datos, se podría caer en casos donde las particiones de datos no contengan datos de alguna de las clases menos representadas, afectando el entrenamiento del modelo.

3. **Bootstrapping**: Acto seguido, se empleará Bootstrapping para entrenar el modelo en esta ocasión con valores los cuales pueden repetirse entre las diferentes muestras, dad la naturaleza del muestro con repetición, para validar que los resultados se mantienen consistentes al momento de realizar las clasificaciones, además de poder reducir el sesgo que podría producirse dado el caso de emplear particiones únicas.

4. **Entrenamiento completo en set de entrenamiento**: Finalmente, se entrenará el modelo en el grupo de entrenamiento completo, como última validación de métricas sin los cambios establecidos por la separación y estratificación o la repetición de datos, además de poder medir el tiempo completo que le tomaría al modelo entrenarse en el tamaño del set de datos.

De cada uno estos pasos se obtendran métricas del entrenamiento como referencia base para el contraste con el set de datos de evaluación del modelo. 

### Separación de los datos en grupos de entrenamiento y prueba

In [21]:
from sklearn.model_selection import train_test_split

# Separando datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

### Cross-validation

A continuación se utilizará el proceso de cross-validation para evaluar cuál modelo tendrá un mejor rendimiento frente a datos desconocidos. Se tomará la decisión partir de la exactitud promedio que tenga cada modelo, es decir, qué tan frecuentemente cada modelo clasifica correctamente en los pliegues de validación.

Dado que los datos se encuentran desbalanceados: la clase de pacientes sanos está sobrerepresentada en comparación con los pacientes pre-diabéticos, y los diabéticos; se ha optado utilizar un muestreo estratificado, con el cual se preservará las proporciones de clases en cada pliegue de entrenamiento.

Para este caso se utilizará el iterador <a href="https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold">Stratified k-fold</a> de la librería scikit-learn utilizando los siguientes parámetros:
- *<b>n_splits</b>*: Indica el número k de veces que se separará los datos en diferentes "folds" siendo k-1 folds los utilizados para el entrenamiento y el último para validación. En otras palabras, se puede entrenar k veces al modelo utilizando en cada iteración diferentes sets de entrenamiento y de validación.
- *<b>shuffle</b>*: Indica si el set de datos se reordenará de manera aleatoria antes de realizar la separación. Esto es de utilidad para evitar que se genere un sesgo debido al orden de los datos.
- *<b>random_state</b>*: Para que los resultados sean reproducibles.

In [36]:
from sklearn.model_selection import StratifiedKFold, cross_validate

# Iterador para validación cruzada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Función para obtener los scores de validación cruzada
def get_cv_scores(model, X, y, cv):
    # Lista de métricas de evaluación
    scoring = ['accuracy', 'f1_macro', 'recall_macro', 'roc_auc_ovr']

    results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=True, error_score='raise')
    
    scores = {}
    # Mostrar resultados
    for metric in scoring:
        scores[metric] = results['test_' + metric]
    return scores

Además, se gráficara cada una de las métricas obtenidas, mostrando lo siguiente: 

* El eje *x* o u horizontal mostrará el rango ajustado a los valores obtenidos a lo largo de las pruebas con cada una de las particiones.
* El eje *y* o vertical mostrará la frecuencia con que dicho valor ha aparecido a lo largo del entrenamiento
* La linea que divide el gráfico de forma vertical marcará la media obtenida del indicador en cuestión, su valor también aparecerá en la leyenda del gráfico para facilitar su interpretación. 

In [22]:
# Función para graficar la distribución de los scores de exactitud
def graph_cv_scores(accuracy_scores, mean_score, metric_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(accuracy_scores, kde=True, bins=30, edgecolor='black', alpha=0.7)

    # Añadir líneas para los percentiles del 95% de IC
    plt.axvline(mean_score, color='black', linestyle='-', label=f'Mean: {mean_score:.4f}')
    
    # Etiquetas y título
    plt.title(f"Distribución de los Scores de {metric_name} (cross-validation)", fontsize=16)
    plt.xlabel(f"{metric_name}", fontsize=14)
    plt.ylabel("Frecuencia", fontsize=14)

    # Leyenda
    plt.legend()

    # Mostrar la gráfica
    plt.show()

### Métricas y generación de intervalos de confianza con bootstrap

A continuación se generan los intérvalos de confianza para cada modelo mediante el método de bootstrap. El método de bootstrap consiste en el remuestreo para estimar métricas en una población a partir de muestras con reemplazo del set de datos disponible.

#### ¿Cómo funciona?
1. De un set de datos grandes se crea una muestra con datos que pueden ser repetidos. Por ejemplo, de un set de 100 datos se toma una muestra de 80 datos, se escoge el dato en la posición i, sin embargo no se elimina de la fuente de datos por lo cual puede volver a ser escogido.
2. Una vez hecha la muestra, se obtiene la métrica de interés de esta. Por ejemplo, si es de interés obtener la media de la población, se obtendría la media de la muestra y se guarda.
3. Se repiten los pasos 1 y 2 hasta un número de iteraciones predefinido.
4. Una vez obtenidas las métricas para cada muestra, se saca el promedio de las métricas.

#### ¿Cómo se aplica en un modelo de aprendizaje automático?
Se repiten exactamente los mismos pasos mencionados anteriormente, sin embargo, se debe tomar en cuenta la siguiente variación: 
- Para el entrenamiento del modelo se utiliza la totalidad de los datos dentro de la muestra, sin embargo, para la evaluación del modelo se debe apartar otro set de datos conocido como out-of-bag u oob.

#### ¿Cómo calcular los intervalos de confianza?
Para obtener los intérvalos de confianza se define un nivel de confianza, por ejemplo: 95%; y se calculan los percentiles dentro de las métricas obtenidas a partir de bootstrap. Los percentiles obtenidos representan los límites inferior y superior del intérvalo. Por ejemplo, si se escoge un nivel de confianza del 95% y se obtienen los siguientes límites del intérvalo de confianza para la métrica de exactitud: [0.84, 0.87] se puede concluir que el 95% de las veces que se ejecute ese modelo tendrá una exactitud entre 84% y 87%.

#### Beneficios
- <b>Permite medir la estabilidad del modelo:</b> Cuando se realiza submuestreo bootstrap, se entrena y evalua el modelo en diferentes subconjuntos de datos. Esto permite ver cómo varía la métrica del modelo cuando se entrena en diferentes muestras de datos. Si el intervalo de confianza es estrecho, significa que la métrica del modelo es bastante estable, lo cual es deseable. Si es amplio, indica que el modelo puede estar siendo sensible a variaciones en los datos y que el rendimiento podría no ser consistente.
- <b>Justificación de selección de modelo:</b> Si el intervalo de confianza de un modelo es mucho más estrecho o más alto en comparación con otros modelos, se puede tener una base más sólida para justificar por qué ese modelo es el mejor.
- <b>Evaluación de la generalización del modelo:</b> El bootstrap permite evaluar cómo el modelo puede generalizar a datos no vistos, ya que se entrena con subconjuntos aleatorios de datos. Si el intervalo de confianza es relativamente pequeño y alto, indica que el modelo tiene un buen rendimiento general y puede generalizar bien, es decir, no está sobreajustado (overfitting).


**Dentro del desarrollo del proyecto, se empleará un intervalo de confianza del 95%**

In [43]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score

# Función para obtener las métricas de evaluación y los límites del intérvalo de confianza
def get_bootstrap_metrics_ci(n_iterations, sample_size, X, y, model):
    # Almacenar resultados de cada métrica
    metrics_results = {
        "accuracy": [],
        "sensitivity": [],  # Sensitivity = Recall para la clase positiva
        "auc": [],
        "f1": [],  # F1-score
    }
    
    indices = np.arange(X.shape[0])

    for i in range(n_iterations):
        np.random.seed(42 + i)
        
        # Selección bootstrap por índices
        bootstrap_indices = np.random.choice(indices, size=sample_size, replace=True)
        oob_indices = np.setdiff1d(indices, bootstrap_indices)

        # Conjunto bootstrap y OOB
        X_ = X[bootstrap_indices]
        y_ = y[bootstrap_indices]
        oob_x = X[oob_indices]
        oob_y = y[oob_indices]

        # Entrenando modelo
        model.fit(X_, y_)

        # Predicciones
        y_pred = model.predict(oob_x)

        # Calcular métricas
        metrics_results["accuracy"].append(accuracy_score(oob_y, y_pred))
        metrics_results["sensitivity"].append(recall_score(oob_y, y_pred, average="macro"))
        metrics_results["f1"].append(f1_score(oob_y, y_pred, average="macro"))
        
        # Manejo de predict_proba para AUC multiclase
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(oob_x)  # Probabilidades para todas las clases
            
            # Calcular AUC multiclase
            try:
                auc_score = roc_auc_score(oob_y, y_pred_proba, multi_class="ovr", average="macro")
                metrics_results["auc"].append(auc_score)
            except ValueError as e:
                print(f"Error en AUC multiclase: {e}")
                metrics_results["auc"].append(None)
        else:
            y_pred_proba = None
            metrics_results["auc"].append(None)

    # Convertir las métricas a un DataFrame para facilidad de manejo
    metrics_df = pd.DataFrame(metrics_results)
    
    # Calcular intervalos de confianza para cada métrica
    ci_results = {}
    for metric in metrics_results.keys():
        ci_results[metric] = {
            "mean": np.mean(metrics_df[metric]),
            "lower_bound": np.percentile(metrics_df[metric].dropna(), 2.5),  # Excluye None
            "upper_bound": np.percentile(metrics_df[metric].dropna(), 97.5)
        }

    return metrics_df, ci_results

Además, se gráficara cada una de las métricas obtenidas, al igual que en Cross Validation siguiendo un esquema muy similar, mostrando lo siguiente: 

* El eje *x* o u horizontal mostrará el rango ajustado a los valores obtenidos a lo largo de las pruebas con cada una de las particiones.
* El eje *y* o vertical mostrará la frecuencia con que dicho valor ha aparecido a lo largo del entrenamiento
* La linea que divide el gráfico de forma vertical marcará la media obtenida del indicador en cuestión, su valor también aparecerá en la leyenda del gráfico para facilitar su interpretación. 

In [44]:
# Función para graficar la distribución de los scores de exactitud
def graph_bootstrap_ci(accuracy_scores, lower_bound, upper_bound, mean_score, metric_name):
    plt.figure(figsize=(10, 6))
    sns.histplot(accuracy_scores, kde=True, bins=30, edgecolor='black', alpha=0.7)

    # Añadir líneas para los percentiles del 95% de IC
    plt.axvline(lower_bound, color='red', linestyle='--', label=f'2.5% Percentile: {lower_bound:.4f}')
    plt.axvline(upper_bound, color='green', linestyle='--', label=f'97.5% Percentile: {upper_bound:.4f}')
    plt.axvline(mean_score, color='black', linestyle='-', label=f'Mean: {mean_score:.4f}')
    
    # Etiquetas y título
    plt.title(f"Distribución de los Scores de {metric_name} (Bootstrap)", fontsize=16)
    plt.xlabel(f"{metric_name}", fontsize=14)
    plt.ylabel("Frecuencia", fontsize=14)

    # Leyenda
    plt.legend()

    # Mostrar la gráfica
    plt.show()

## Esquema de Prueba de Funcionamiento del modelo

Para poder evaluar el resultado final del entrenamiento de cada uno de los modelos y obtener métricas reales del funcionamiento del modelo, estos serán entrenados con los datos reservados para evaluación, y se procederá a un análisis de las métricas obtenidas por los modelos para poder determinar cual sería el mejor a seleccionar. 